In [1]:
import numpy as np
from glob import glob
import os, sys
import re

In [2]:
topDirName = '/home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training'

In [19]:
#Assumes current working directory is ~/src/YPB-AI
allSamples = np.array(glob(topDirName + '/*'))
os.system('mkdir ' + topDirName + '/Absent')
os.system('mkdir ' + topDirName + '/Present')

for sample in allSamples: 
    if 'Negative' in sample or 'Sat' in sample:
        cmdString = "mv " + sample + " " + topDirName + "/Absent"
    else:
        cmdString = "mv " + sample + " " + topDirName + "/Present"
    os.system(cmdString)

In [6]:
digestedSample

['',
 'home',
 'ubuntu',
 'src',
 'YPB-AI',
 'beforeSplit',
 'AlexNet',
 'Training',
 'Present',
 '6_Sample2_Dark',
 'ProtectCodeScannerResults_08-Mar-2019_17-15-26']

In [20]:
### Find 'vestigial' directories to be deleted after unpacking.
presentVestDirs = np.array(glob(topDirName + '/Present/*'))
absentVestDirs = np.array(glob(topDirName + '/Absent/*'))

# Unpack the JPG photo files into identifiable directories.
allPresentSamples = np.array(glob(topDirName + '/Present/*/*'))
for sample in allPresentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[10])
    newDir = digestedSample[9] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    
allAbsentSamples = np.array(glob(topDirName + '/Absent/*/*'))
for sample in allAbsentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[10])
    newDir = digestedSample[9] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    
    
# Delete vestigial directories.
for direc in presentVestDirs:
    os.system('rm -rf ' + direc)
    
for direc in absentVestDirs:
    os.system('rm -rf ' + direc)

In [21]:
# Get the filenames.
# imgSeqsToCorrect = os.popen('cd ~/src/YPB-AI/; find -name "*IMG_9999.JPG"').read()
imgSeqsToCorrect = os.popen('cd ~/src/YPB-AI/beforeSplit/AlexNet/Training/; find -name "*IMG_9999.JPG"').read()
# Split into unique entities.
imgSeqsToCorrect = re.split('\n',imgSeqsToCorrect)
# Drop the last entry, it's empty.
imgSeqsToCorrect.pop();

# Change any files that don't start with 'IMG_9' (i.e., any files that come after the wraparound) to start with 'J'.
# This will mean that when they are sorted, they will come correctly after the wraparound.
for sample in imgSeqsToCorrect:
    digestedSample = re.split('/',sample)
    # pathToDir = os.path.join(digestedSample[0],digestedSample[1],digestedSample[2],digestedSample[3])
    pathToDir = os.path.join(topDirName,digestedSample[1],digestedSample[2])
    # pathToDir = topDirName + '/' + digestedSample[0] + '/' + digestedSample[1] + '/' + digestedSample[2] + '/' + digestedSample[3]
    # try:
    filesInDir = os.listdir(pathToDir)
    for i in range(len(filesInDir)):
        if 'IMG_9' not in filesInDir[i]:
            newFile = filesInDir[i].replace('I','J',1)
            # cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
            cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
            # cmdString = 'mv ' + '/' + filesInDir[i] + '/' + ' ' + pathToDir + '/' + newFile
            # print(cmdString)
            os.system(cmdString)
    # except Exception as e:
    #     print(e)

In [5]:
digestedSample

['.', 'Absent', '6S_Negative_Dark_14-Mar-2019_09-53-30', 'IMG_9999.JPG']

# Use AlexNet to transform image sequences to a pickle file and organize the results.

In [3]:
import torch
import torchvision.models as models
from PIL import Image
import pickle

In [4]:
# Import the CNN model (AlexNet) and move it to CUDA if available.
model = models.alexnet(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
del model.classifier

if torch.cuda.is_available():
    model.to('cuda');

In [15]:
# This part is for converting into image. Not using it now.
# topdir = topDirName
# ndx = 0
# topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']

# # Calculate the total number of stacks to transform.
# numStacks = 0
# for classDir in topdirs:
#     thisClassDir = os.path.join(topdir,classDir)
#     directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
#     numStacks += len(directories)

# for classDir in topdirs:
#     thisClassDir = os.path.join(topdir,classDir)
#     directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']


#     # Do the transformation.
#     for photoDir in directories:

#         thisPhotoDir = os.path.join(thisClassDir, photoDir)
#         ##pin pin fix##

# #             print(thisPhotoDir)
# #             print(os.path.isfile(photoDir))
# #             continue

#         if not os.path.isdir(thisPhotoDir):
#             continue

#         ## end ##
#         files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
#         print("Transforming image stack {}".format(thisPhotoDir))
#         files.sort()
#         output = []
#         # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
#         for file in files:
#             thisFile = os.path.join(thisPhotoDir,file)
#             imgArr = np.float32(np.array(Image.open(thisFile).resize((224,224))).reshape([1,3,224,224]))
#             imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
#             imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
#             imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
#             output.append(imgArr)

#         FinalOutput = torch.FloatTensor(output)
#         # for i in range(len(files)):
#         #    FinalOutput[i] = output[i]
#         pathStr = thisPhotoDir + '.pickle'
#         with open(pathStr,'wb') as f:
#             pickle.dump(FinalOutput,f)
#             ndx += 1
#             print("{}% complete.".format(ndx/numStacks*100))
    

Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/6S_Negative_Sample3_Light_16-Apr-2019_15-59-47
0.008768084173608068% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/XS_Sat2_07-Jun-2019_20-13-37
0.017536168347216136% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_13-17-37
0.026304252520824196% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/SE_Negative_Dark_15-Mar-2019_15-37-50
0.03507233669443227% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/XS_Sat_17-Apr-2019_11-36-09
0.04384042086804033% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_14-11-36
0.05260850504164839% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSp

0.4734765453748356% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/XS_Sat2_07-Jun-2019_17-28-05
0.48224462954844366% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/XS_Sat_17-Apr-2019_11-36-24
0.49101271372205174% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/SE_Negative_Dark_15-Mar-2019_16-40-40
0.4997807978956598% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/7_Negative_Dark_13-Mar-2019_16-51-57
0.5085488820692678% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/6S_Sat_03-Jun-2019_14-19-25
0.5173169662428759% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/XS_Sat2_07-Jun-2019_18-38-34
0.526085050416484% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/7_Neg

KeyboardInterrupt: 

Define the function that will transform model output into a pickle file.

In [5]:
def transform_to_pickle(model,topdir):
    model.eval()
    ndx = 0
    topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']
    
    # Calculate the total number of stacks to transform.
    numStacks = 0
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        numStacks += len(directories)
    
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        
        
        # Do the transformation.
        for photoDir in directories:
            
            thisPhotoDir = os.path.join(thisClassDir, photoDir)
            ##pin pin fix##
            
#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue
            
            if not os.path.isdir(thisPhotoDir):
                continue

            ## end ##
            files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
            print("Transforming image stack {}".format(thisPhotoDir))
            files.sort()
            output = []
            # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
            for file in files:
                thisFile = os.path.join(thisPhotoDir,file)
                imgArr = np.float32(np.array(Image.open(thisFile).resize((256,256))).reshape([1,3,256,256]))
                imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
                imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
                imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
                if torch.cuda.is_available():
                    output.append(model(torch.cuda.FloatTensor(imgArr)))
                else:
                    output.append(model(torch.FloatTensor(imgArr)))

            FinalOutput = imgArr
            pathStr = thisPhotoDir + '.pickle'
            with open(pathStr,'wb') as f:
                pickle.dump(FinalOutput,f)
                ndx += 1
                print("{}% complete.".format(ndx/numStacks*100))

Do the transformation

In [12]:
# PyTorch pre-trained AlexNet somehow didn't work
import torch.nn as nn
class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [6]:
transform_to_pickle(model,topDirName)

Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/AlexNet/Training/Absent/6S_Negative_Sample3_Light_16-Apr-2019_15-59-47


AttributeError: 'AlexNet' object has no attribute 'classifier'

Clean up the directory, removing everything but the pickle files.

In [22]:
# Assemble all the pickle files and remove the rest.
classDirs = [direc for direc in os.listdir(topDirName) if direc[0] is not '.']
for classDir in classDirs:
    newDir = os.path.join(topDirName,classDir)
    fileDirs = [direc for direc in os.listdir(newDir) if direc[0] is not '.' and not direc.lower().endswith('.pickle')]
    for fileDir in fileDirs:
        pickleFileDir = os.path.join(newDir,fileDir)
        os.system('rm -rf ' + pickleFileDir)

# Split the directory into train/test/validation sets

In [23]:
import random

In [24]:
def split_folders(inputDir, outputDir, trainValTestRatio, seed):
    # Assume inputDir is organized as 'inputDir/class/topFolder/photos'
    classes = os.listdir(inputDir)
    # Exclude folders where '.' is the first character.
    classes = [directory for directory in classes if directory[0] != '.']
    numClasses = len(classes)
    # Create output directories
    os.system('mkdir ' + outputDir)
    os.system('mkdir ' + outputDir + '/train')
    os.system('mkdir ' + outputDir + '/valid')
    os.system('mkdir ' + outputDir + '/test')
    # Pull files and put into the appropriate folders.
    for directory in classes:
        files = os.listdir(os.path.join(inputDir, directory))
        files.sort()
        random.seed(seed)
        random.shuffle(files)
        numFiles = len(files)
        # Get the indices.
        ndxAmts = np.intc((np.round(trainValTestRatio / np.sum(trainValTestRatio) * numFiles)))
        numNdxes = np.sum(ndxAmts)
        if numNdxes != numFiles:
            ndxAmts[0] += numFiles-numNdxes
        trainFiles = files[0:ndxAmts[0]]
        valFiles = files[ndxAmts[0]:ndxAmts[0]+ndxAmts[1]]
        testFiles = files[ndxAmts[0]+ndxAmts[1]:]
        # Put the training directories in the appropriate folder
        # First make the class directories.
        os.system('mkdir ' + outputDir + '/train/' + directory)
        os.system('mkdir ' + outputDir + '/valid/' + directory)
        os.system('mkdir ' + outputDir + '/test/' + directory)
        for file in trainFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/train/' + directory + '/' + file)
        for file in valFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/valid/' + directory + '/' + file)
        for file in testFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/test/' + directory + '/' + file)

In [25]:
finalDirName = 'Split_Data2_29_02_20'

In [27]:
# for idx in range(5):
#     split_folders(topDirName, finalDirName_root + str(idx),[0.8,0.1,0.1],1000 + idx)
finalDirName

'Split_Data2_17_01_20'

In [28]:
split_folders(topDirName, finalDirName,[0.8,0.1,0.1],7778)